In [1]:
import numpy as np
import glob, time
from astropy.table import Table
from astropy.io import ascii

In [8]:
t = ascii.read('movie_ratings.csv')
nUser = np.max(t['userId']) # total number of users
nMovie = np.max(t['movieId']) # total number of movies

In [52]:
R = np.zeros(shape = (nUser, nMovie))

for nrow in np.arange(np.size(t)):
    i = t[nrow][0] - 1 # the userId
    j = t[nrow][1] - 1 # the movieId
    R[i][j] = t[nrow][2] # the rating

In [36]:
R

array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [64]:
class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

#         # Initialize the biases
#         self.b_u = np.zeros(self.num_users)
#         self.b_i = np.zeros(self.num_items)
#         self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
#             # Update biases
#             self.b_u[i] += self.alpha * e * self.b_u[i]
#             self.b_i[j] += self.alpha * e * self.b_i[j]

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * e * self.Q[j, :]
            self.Q[j, :] += self.alpha * e * self.P[i, :]

#             # Update biases
#             self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
#             self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

#             # Update user and item latent feature matrices
#             self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
#             self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
#         prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        prediction = self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.P.dot(self.Q.T)

In [65]:
mf = MF(R, K = 300, alpha = 0.01, beta = 0.01, iterations = 20)

In [66]:
mf.train()

Iteration: 10 ; error = 245.7600
Iteration: 20 ; error = 167.7297


[(0, 1127.0150942989414),
 (1, 536.6652222748211),
 (2, 397.88150707584094),
 (3, 342.16408050693343),
 (4, 311.2101279754552),
 (5, 291.2605279157728),
 (6, 276.471622245819),
 (7, 265.3045807202362),
 (8, 254.73334962337853),
 (9, 245.759954768478),
 (10, 237.74153140730314),
 (11, 229.75229930596527),
 (12, 222.46522103888395),
 (13, 214.6292189810857),
 (14, 207.08003765660735),
 (15, 199.15938545220712),
 (16, 191.35131681999204),
 (17, 183.09691432950606),
 (18, 174.95828422106587),
 (19, 167.7297357152622)]

In [67]:
mf.full_matrix()

array([[ 4.36695266e+00,  4.00559506e+00,  4.10619084e+00, ...,
        -8.49217016e-03,  7.52232842e-04,  4.41385278e+00],
       [ 3.84834392e+00,  3.47825110e+00,  3.33656029e+00, ...,
        -6.20060136e-03,  6.51501764e-04,  3.52057489e+00],
       [ 2.16838208e+00,  2.20749198e+00,  2.12694066e+00, ...,
        -3.27156007e-03, -1.90274535e-03,  2.49517628e+00],
       ...,
       [ 1.78071871e+00,  1.69946692e+00,  1.80354928e+00, ...,
         1.08751826e-02,  3.07155812e-02,  3.35708554e+00],
       [ 3.45548535e+00,  3.10507379e+00,  3.01864292e+00, ...,
        -4.54302299e-03,  2.52447733e-03,  3.12720564e+00],
       [ 5.06295135e+00,  3.37349689e+00,  3.45282615e+00, ...,
         1.39257538e-03, -7.32187900e-03,  4.06822885e+00]])

In [68]:
mf.mse()

167.7297357152622